$$
\large \textbf{Problema 02}
$$

Se tiene una barra delgada de aluminio, originalmente a $0\:°C$, cuyos extremos se someten repentinamente a temperaturas fijas. El extremo izquierdo se mantiene a una temperatura constante de $100\:°C$ y el derecho a $50\:°C$. Utilice los métodos explícito e implícito (Crank-Nicolson) de diferencias finitas para calcular la evolución de la distribución de temperaturas en función del tiempo. Tome que la barra tiene $10\:cm$ de longitud y $k\:=\:0.835\:\frac{cm^2}{s}$ (aluminio). Haga los cálculos para $Δx\:=\:2\:cm$ y $Δt\:=\:0.1\:s$. Repítalos para $Δt\:=\:5\:s$ y compare sus resultados. Analice el comportamiento de la solución para los distintos regímenes.
***

In [ ]:
import numpy as np                # Librería de álgebra lineal
import matplotlib.pyplot as plt   # Librería de gráficos